<a href="https://colab.research.google.com/github/DavidScience/AB_InBev_/blob/main/%5B5%5D_Desaf%C3%ADo_AB_InBev_Output_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [0] Librerías necesarias

In [1]:
# Conectamos Google Drive con Google Colab
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
# Librerías necesarias para el desarrollo
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
import multiprocessing as mp
from joblib import Parallel, delayed
import pickle

## [1] Lectura de datos

In [3]:
# Lectura de datos preparados para el entrenamiento del algoritmo
output_sr = pd.read_csv('/gdrive/My Drive/AB_InBev/datos/output_sr.csv')

In [4]:
# Lectura de datos preparados para el entrenamiento del algoritmo
data_scoring_df = pd.read_csv('/gdrive/My Drive/AB_InBev/datos/data_scoring_df.csv')

In [5]:
# Inspeccionamos los primeros registros
output_sr.head(5)

,Account_id,Product_id
0,33214647,17468
1,33214647,16582
2,33214647,14014
3,33214647,8222
4,33214647,8352


In [6]:
# Inspeccionamos los primeros registros
data_scoring_df.head(5)

,Date,product,Quantity,Quantity_lag_1,Quantity_lag_2,Quantity_lag_3,Quantity_lag_4,Quantity_lag_5,Quantity_lag_6,Quantity_lag_7,...,Quantity_lag_22,Quantity_lag_23,Quantity_lag_24,Quantity_lag_25,Quantity_lag_26,Quantity_lag_27,Quantity_lag_28,Quantity_lag_29,Quantity_lag_30,y_pred
0,2022-07-30,40635321_17466,1173,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,656.867577
1,2022-07-30,36756690_14016,1173,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,951.411464
2,2022-07-30,33222003_15354,1122,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,790.0,-1.0,-1.0,772.467871
3,2022-07-30,36615003_15354,1122,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,656.867577
4,2022-07-30,37607805_15354,1100,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,656.867577


In [7]:
# Creamos el campo que concatena id de cliente y producto
output_sr['product'] = output_sr['Account_id'].astype('str')+'-'+output_sr['Product_id'].astype('str')

In [12]:
# Incluimos al campo de recomendaciones la cantidad a comprar
output = pd.concat([output_sr, data_scoring_df[['product','y_pred']]], axis=1, join='outer')

In [14]:
# Filtramos y renombramos campos
output=output[['Account_id','Product_id','y_pred']]
output.rename(columns = {'y_pred':'Recommended_quantity'}, inplace = True)

<ipython-input-14-0e719f346b8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output.rename(columns = {'y_pred':'Recommended_quantity'}, inplace = True)


In [16]:
# Inspeccionamos los primeros registros
output.head(10)

,Account_id,Product_id,Recommended_quantity
0,33214647,17468,656.867577
1,33214647,16582,951.411464
2,33214647,14014,772.467871
3,33214647,8222,656.867577
4,33214647,8352,656.867577
5,33217422,8328,671.351995
6,33217422,8320,686.173639
7,33217422,8356,656.867577
8,33217422,8334,1008.011796
9,33217422,8230,906.632622


In [17]:
# Exportamos el output final del proceso
output.to_csv('/gdrive/My Drive/AB_InBev/datos/output.csv',index=False)